In [ ]:
# Copyright (C) 2019  Communication Network Security Laboratory, SSU, Seoul

# This program is free software; you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation; either version 2 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.

# LibAndAssets

## What is this code ? 
This code are used to generate the list of packer signature based on mutual_dict.

To the effective of our method, applied the train/test packer dataset that was splitted by DEX Analysis experiments.

In [1]:
import os
import configparser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# CONFIG ARGURMENTS
ASSETS_DIR = "dataset/kaggle/assets/"
BASE_DIR = "dataset/kaggle/"

# STEP 1: COLLECT DATASET AND TRANSFORM

THIS STEP HAS BEEN RUN AND EXPORTED AS PICKLE FILE. CHANGE THE MARKDOWN SECTIONS THAT COME WITH "CODE" FOR RERUN

In [3]:
### CODE - DEFINE CLASS 
class Assets_FeatureModel(object):
    """Model for classification."""
    def commonizePacker(self, df):                                                                                                                                                                                                                                                                                                                                                                        
        df['packerizer'] = df['packer'].apply(self.packerlizer)                      
        return df
    
    def hasExt(self, x):
        if "." in x:
            return True
        return False
    
    def hasValidData(self, x):
        if "." in x:
            valid_data = ['data', 'so', 'jar', 
                          'dat', 'bin', 'zip',
                          'jpg'
                         ]
            ext = x.split(".")[-1]
            for d in valid_data:
                if d == ext:
                    return True
            return False
        else:
            return False
        
    def hasValidDataType(self, x):
        valid_data = ['data', 'elf', 'zip']
        print(x)
        for d in valid_data:
            if d == x:
                return True
        return False
              
    def refineDF(self, df):
        #df = df[~df['name'].str.contains(".png")]
        #df = df[~df['name'].str.contains(".jpg")]
        #df = df[~df['name'].str.contains(".ogg")]
        #df = df[~df['name'].str.contains(".RSA")]
        #df = df[~df['name'].str.contains(".lua")]
        #df = df[~df['name'].str.contains(".xml")]
        #df = df[~df['name'].str.contains(".obm")]
        #df = df[~df['name'].str.contains(".btb")]
        # df = df[df['type'].apply(self.hasValidDataType) == True] # BAD MODEL
        df = df[df['name'].apply(self.hasValidData) == True]
        #df = df[df['name'].apply(self.hasExt) == True]

        return df

    def vector_hash(self, x):
        hash_x = hash("-".join(str(e) for e in x))
        return hash_x
        
    def createHashVector(self, df):
        df['vector_hash'] = df['vector'].apply(self.vector_hash)      
        return df
    
    def packerlizer(self, x):
        unique_packers = [                                                                              
            'Ijiami', 'Bangcle', 'Qihoo 360', 'APKProtect',                                             
            'Jiagu'                                                                                   
        ]
        for i in unique_packers:
            if i in x:
                return i
        return x
        


GET DATASET

In [4]:
# CODE - THIS STEP COLLECT PACKER AND NORMAL DATASET WHICH COLLECT BY OUR SCRIPT (PRE-PROCESSING STAGE)

packer_df = pd.DataFrame()
normal_df = pd.DataFrame()
for f in os.listdir(ASSETS_DIR):
    if all(x in f for x in ['df_assets_packer']):
        df = pd.read_json(os.path.join(ASSETS_DIR, f))
        packer_df = packer_df.append(df)
    if all(x in f for x in ['df_assets_normal']):
        df = pd.read_json(os.path.join(ASSETS_DIR, f))
        normal_df = normal_df.append(df)

# CODE - COMBINED WITH DATASET FROM DEX ANALYSIS

#### INTEGRATED ARE BASED ON DEX ANALYSIS DATASET (ALREADY UNDER-SAMPLING)
INTEGRATED_FILE = os.path.join(BASE_DIR,"integrated_df.pickle") 
integrated_df = pd.read_pickle(INTEGRATED_FILE)
saved_packer_df = integrated_df[integrated_df['packer_label'] != 0]

##### GET Intersect of md5
md5_packer = np.intersect1d(saved_packer_df.md5, packer_df.app_md5).tolist()
print("Number of packer in test: %d" % len(md5_packer))

# CODE - CHECK FOR PACKER ASSETS AND MAKE NEW PACKER DF (SAME)

#### Make NEW FOR IMPLEMENTATIONS, WE DON"T NEED TO SPLIT ANYMORE
i_packer_df = pd.DataFrame()
i_normal_df = normal_df

for md5 in md5_packer:
    i_packer_df = i_packer_df.append(packer_df[packer_df['app_md5'] == md5], ignore_index=True)
    
print("Number of packer assets in test: %d" % len(i_packer_df.index))

### Transforming - Reducing

Create hashes vector based on the packers information

# CODE - TRANSFORM THE PACKER DATASET

# Packerizer and extract valid data
#feature_model = Assets_FeatureModel()
#i_packer_df = feature_model.refineDF(i_packer_df)
#i_packer_df = feature_model.commonizePacker(i_packer_df)
#i_packer_df = feature_model.createHashVector(i_packer_df)


# DEBUG

##### Unique_signature, Test
dup_sig = i_packer_df[i_packer_df['vector_hash'].duplicated()]
uniq_sig = i_packer_df['vector_hash'].unique()
final_packer_df = pd.DataFrame()

#### TAKE A LONG TIME FOR THIS 
for sig in uniq_sig:
    # GET ONE SAMPLE FROM A VECTOR_HASH TO CALCULATE
    final_packer_df = final_packer_df.append(
        i_packer_df[i_packer_df['vector_hash'] == sig].iloc[0], ignore_index=True)
        
final_packer_assets = os.path.join(ASSETS_DIR, "final_packer_assets_27012019.pickle")
final_packer_df.to_pickle(final_packer_assets)


# STEP 2: TRAINING

#### DEBUG - SAVE THE FULL PACKER DATASET TO FILE

#WRONG CODE HERE!, THE ASSETS HAVE TO BE REDUCED AFTER SPLITTED. WE REDUCED THE ASSETS TO IMPROVE TRAINING PROCESS. 
final_packer_assets = os.path.join(ASSETS_DIR, "final_packer_assets_24012019.pickle")
final_packer_assets_df = pd.read_pickle(final_packer_assets)
#END WRONGCODE

#### CODE - Collect packer df from stored file
final_packer_df_path = os.path.join(ASSETS_DIR, "final_packer_df_24012019.pickle")
i_packer_df.to_pickle(final_packer_df_path)

In [5]:
final_packer_df_path = os.path.join(ASSETS_DIR, "final_packer_assets_27012019.pickle")
i_packer_df = pd.read_pickle(final_packer_df_path)

# Collect normal df from stored file
final_normal_df = pd.DataFrame()
for f in os.listdir(ASSETS_DIR):
    if all(x in f for x in ['df_assets_normal']):
        df = pd.read_json(os.path.join(ASSETS_DIR, f))
        final_normal_df = final_normal_df.append(df)
        
# ax = sns.countplot(y="", data=final_packer_df)

####  COLLECT THE REAL TRAIN & TEST PACKER DATASET BASED ON MD5

Get list of train/test packer dataset based on md5 list that shared by DEX analysis process

In [6]:
# TEST WITH SHARED
# SPLITTED_PACKER_FILE = os.path.join(BASE_DIR,"splited_packer_df.pickle")
# splited_packer = pd.read_pickle(SPLITTED_PACKER_FILE)

if os.path.isfile(os.path.join(BASE_DIR, "train_packer_md5.pickle")):
    md5_train = pd.read_pickle(os.path.join(BASE_DIR, "train_packer_md5.pickle"))
if os.path.isfile(os.path.join(BASE_DIR, "test_packer_md5.pickle")):
    md5_test = pd.read_pickle(os.path.join(BASE_DIR, "test_packer_md5.pickle"))
    
md5_train.columns=['app_md5', 'packer']
md5_test.columns=['app_md5', 'packer']

merged_train_df = pd.merge(packer_df, md5_train, how='inner', on=['app_md5']) # Merged to final train set
merged_test_df = pd.merge(packer_df, md5_test, how='inner', on=['app_md5']) # Merged to final test set

print("Total packer for test: %d" % len(merged_test_df.app_md5.unique()))
print("Total packer for train: %d" % len(merged_train_df.app_md5.unique()))



# Calculate vector_hash
feature_model = Assets_FeatureModel()

merged_test_df = merged_test_df.drop(['packer_y'], axis=1)
merged_test_df = merged_test_df.rename(index=str, columns={"packer_x": "packer"})
merged_test_df = feature_model.refineDF(merged_test_df)
merged_test_df = feature_model.commonizePacker(merged_test_df)
merged_test_df = feature_model.createHashVector(merged_test_df)

merged_train_df = merged_train_df.drop(['packer_y'], axis=1)
merged_train_df = merged_train_df.rename(index=str, columns={"packer_x": "packer"})
merged_train_df = feature_model.refineDF(merged_train_df)
merged_train_df = feature_model.commonizePacker(merged_train_df)
merged_train_df = feature_model.createHashVector(merged_train_df)

print("Total test packer after refining: %d" % len(merged_test_df.app_md5.unique()))
print("Total train packer after refining: %d" % len(merged_train_df.app_md5.unique()))



Total packer for test: 196
Total packer for train: 286
Total test packer after refining: 195
Total train packer after refining: 284


#### After Refined

In [7]:
print("Total test assets: %d" % len(merged_test_df.index))
print("Total train assets: %d" % len(merged_train_df.index))

Total test assets: 7599
Total train assets: 13807


#### NOW, GET THE UNIQUE LIST OF ASSETS

In [8]:
#uniq_sig = pd.DataFrame(columns=['vector_hash'])
#uniq_sig['vector_hash'] = merged_train_df['vector_hash'].unique()
#reduced_ptrain_df = pd.merge(merged_train_df, uniq_sig, how='inner', on=['vector_hash'])

reduced_test_df = merged_test_df.drop_duplicates(subset='vector_hash', keep="first")
print("Before remove duplicated: %d" % len(merged_test_df['vector_hash'].index))
print("After remove duplicated: %d" % len(reduced_test_df['vector_hash'].index))

Before remove duplicated: 7599
After remove duplicated: 1347


# NEXT, CALCULATE THE MUTUAL INFORMATION

In [9]:
from collections import Counter
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
import pickle

def _createTask(train, alg, p, threshold):
    print("Training for packer: %s" % p) 
    Xtrain = []
    ytrain = []
    p_df = train[train['packerizer'] == p]
    collect_num = len(p_df.index)
    if collect_num < 10:
        return
    for index, row in p_df.iterrows():
        counts = Counter(row['vector'])
        fvector = dict(counts)
        Xtrain.append(fvector)
        ytrain.append(row['packer'])
    dv = DictVectorizer()
    X_vec = dv.fit_transform(Xtrain)
    feature_scores = mutual_info_classif(X_vec, ytrain)
    if alg == "chi":
        feature_scores = chi2(X_vec, ytrain)[0]
    if alg == "f_classif":
        feature_scores = f_classif(X_vec, ytrain)[0]
    names = []
    for score, fname in sorted(zip(feature_scores, dv.get_feature_names()), reverse=True)[:threshold]:
        names.append(fname)
    return names

def createMutualDict(train, alg="mi", threshold=10):
    import pickle
    if os.path.isfile(os.path.join(BASE_DIR, "mutual_dict_t%d.pickle" % threshold)):
        return pickle.load(open(os.path.join(BASE_DIR, "mutual_dict_t%d.pickle" % threshold), "rb" ))
    from tqdm import tqdm
    import multiprocessing
    mutual_dict = {}
    total_packer = len(train.packerizer.unique())
    pbar = tqdm(total=total_packer)
    for p in train.packerizer.unique():
        result = _createTask(train, alg, p, threshold)
        mutual_dict[p] = result
        pbar.update(1)
    pbar.close()
    pickle.dump(mutual_dict, open( "mutual_dict_t%d.pickle" % threshold, "wb" ))
    return mutual_dict
start_time = time.time()
mutual_dict = createMutualDict(merged_train_df, "mi")
elapsed_time = time.time() - start_time
print("Elapse time: %d" % elapsed_time)


  0%|          | 0/11 [00:00<?, ?it/s]

Training for packer: Unicom SDK Loader


  9%|▉         | 1/11 [00:04<00:42,  4.24s/it]

Training for packer: Ijiami


 18%|█▊        | 2/11 [00:05<00:31,  3.48s/it]

...

Training for packer: Naga


 91%|█████████ | 10/11 [00:14<00:00,  1.03it/s]

Training for packer: UPX (unknown, modified)


100%|██████████| 11/11 [00:15<00:00,  1.04it/s]

Elapse time: 15


In [10]:
# Save mutual dict
print(mutual_dict)

{'Naga': [108, 965, 960, 955, 953, 951, 950, 948, 947, 945], 'Bangcle': [108, 676, 813, 111, 100, 841, 184, 843, 787, 57], 'SecNeo': [957, 951, 946, 944, 934, 932, 930, 927, 925, 921], 'Tencent': [947, 714, 918, 745, 583, 581, 491, 511, 488, 211], 'Qihoo 360': [799, 348, 122, 114, 967, 811, 804, 792, 112, 257], 'DexProtector': [915, 110, 930, 921, 118, 45, 971, 923, 918, 916], 'Ijiami': [97, 51, 50, 46, 55, 56, 54, 53, 52, 103], 'UPX (unknown, modified)': [918, 888, 877, 870, 862, 855, 854, 849, 847, 828], 'Unicom SDK Loader': [136, 126, 80, 72, 179, 93, 978, 333, 152, 403], 'Jiagu': [95, 119, 106, 100, 112, 103, 108, 98, 105, 46], 'APKProtect': [958, 934, 965, 453, 946, 939, 917, 643, 931, 922]}


In [11]:
import queue
import threading

class MultiTask(object):
    """For running multi-task."""

    def __init__(self, task_name, thread_num=10):
        """Init."""
        self.task_name = task_name
        self.tnum = thread_num

    def run(self, input_list):
        """Run task."""
        work = queue.Queue()
        results = queue.Queue()

        for i in range(self.tnum):
            t = threading.Thread(target=self.task_name,
                                 args=(work, results))
            t.daemon = True
            t.start()

        i = 0
        for i in range(len(input_list)):
            work.put(input_list[i])

        work.join()

        i = 0
        out = []
        for i in range(len(input_list)):
            res = results.get()
            out.append(res)
        return out

In [12]:
import operator
import json
from tqdm import tqdm
CACHE_APP_DIR = os.path.join(ASSETS_DIR, "cache")

class MutualInformationClassification(object):
    def __init__(self):
        self.cache = {}
        self.appCache = {}
        self.cacheFlg = 1
        self.checkType = 1 # 0 is assets_check, 1 is app_check
        
    def setCache(self):
        self.cacheFlg = 1
        return
    
    def unsetCache(self):
        self.cacheFlg = 0
        return
    
    def getCache(self):
        return self.cache
    
    def setCache(self, json):
        self.cache = json
        return 
    
    def clearCache(self):
        self.cache = {}
        return

    def storeCache(self):
        # Store cache result to json
        last_num = 0
        for f in os.listdir(ASSETS_DIR):
            if "cache_assets" in f:
                # Store cache_assets named with: cache_assets_<num>
                cache_num = int(f.split("_")[-1])
                if cache_num > last_num:
                    last_num = cache_num

        last_num += 1
        saved = os.path.join(ASSETS_DIR, "cache_assets_%d" % last_num)
        with open(saved, 'w') as fp:
            json.dump(self.cache, fp)
        self.clearCache()
        
    def loadAppCache(self, md5):
        data = {}
        cache_app = "cache_app_%s" % md5
        for f in os.listdir(CACHE_APP_DIR):
            if cache_app in f:
                saved = os.path.join(CACHE_APP_DIR, f)
                with open(saved, 'r') as fp:
                    ds = json.load(fp)
                    data.update(ds)
        self.appCache = data
        print("Loaded %d vector hash for app: %s " % (len(data), md5))
        return self.appCache
    
    def storeAppCache(self, md5):
        # Store cache result to json
        saved = os.path.join(CACHE_APP_DIR, "cache_app_%s" % md5)
        if not os.path.isfile(saved):
            return
        with open(saved, 'w') as fp:
            json.dump(self.appCache, fp)
        print("Stored %d vector hash to app: %s " % (len(self.appCache), md5))
        self.clearAppCache()
            
    def clearAppCache(self):
        self.appCache = {}
        return
    
    def loadCache(self):
        data = {}
        for f in os.listdir(ASSETS_DIR):
            if "cache_assets" in f:
                saved = os.path.join(ASSETS_DIR, f)
                with open(saved, 'r') as fp:
                    ds = json.load(fp)
                    data.update(ds)
        self.cache = data
        print("Loaded %d vector hash" % len(data))
        return self.cache
        
    def taskMiCal(self, in_queue, out_queue):
        """Threading work."""
        while True:
            inp = in_queue.get()
            # process
            result = self.taskMiCal_t(inp)
            out_queue.put(result)
            in_queue.task_done()
        return None
    
    def taskMiCal_t(self, input_list):
        fvector = input_list['fvector']
        packer = input_list['packer']
        tested  = input_list['tested']
        m = input_list['m'] 
        threshold = input_list['threshold']
        ytrain = []
        ytrain.append(tested)
        comXTrain = []
        comXTrain.append(fvector)
        counts2 = Counter(m)
        fvector2 = dict(counts2)
        comXTrain.append(fvector2)
        ytrain.append(packer)
        dv = DictVectorizer()
        X_vec = dv.fit_transform(comXTrain)
        feature_scores = mutual_info_classif(X_vec, ytrain)
        matched = 0
        for score, fname in sorted(zip(feature_scores, dv.get_feature_names()), reverse=True)[:threshold]:
            if score != 0.6931471805599453:
                matched += 1
        if matched > 0:            
            return packer
        else:
            return None
    
    def checkRow(self, app_df, mutual_dict, threshold):
        # import multiprocessing
        # pool = multiprocessing.Pool(processes =  len(mutual_dict))
        output = []
        asset_vector = []
        packer_vector = []
        type_vector = []
        print("Check %d assets from app" % len(app_df.index))
        for index, row in app_df.iterrows():
            # Check hashes
            data_type = row['type']
            assets = row['name']
            if self.cacheFlg:
                if row['vector_hash'] in self.cache:
                    predict = self.cache[row['vector_hash']]
                    output.append("%s:%s:%s:%s" % (assets, data_type, predict, row['vector_hash']))
                    continue
                if row['vector_hash'] in self.appCache:
                    predict = self.appCache[row['vector_hash']]
                    output.append("%s:%s:%s:%s" % (assets, data_type, predict, row['vector_hash']))
                    continue
            tested = row['packer']
            counts = Counter(row['vector'])
            fvector = dict(counts)
            matched_str = None
            packer_list = {}
            for p,m in mutual_dict.items():
                # Compare with train
                #result = pool.apply_async(taskMiCal, (fvector,tested,m, threshold,))
                inp = {'fvector': fvector, 'packer': p,
                       'tested': tested, 'm': m, 'threshold':threshold}
                result = self.taskMiCal_t(inp)
                if result is not None:
                    if result in packer_list:
                        packer_list[result] += 1
                    else:
                        packer_list[result] = 1
            if len(packer_list) > 0:
                predict = max(packer_list.items(), key=operator.itemgetter(1))[0]
                output.append("%s:%s:%s:%s" % (assets, data_type, predict, row['vector_hash']))
                if self.cacheFlg:
                    if self.checkType == 0: # cache assets
                        self.cache[row['vector_hash']] = predict
                        if len(self.cache) % 10000 == 0:
                            self.storeCache()
                    else:
                        self.appCache[row['vector_hash']] = predict
                # input_list.append(inp)
            #print("Input list: %d" % len(input_list))
            #task_thread = MultiTask(taskMiCal)
            #result = task_thread.run(input_list)
            #if result is not None:
            #    packer_str = ""
            #    for res in result:
            #        if res is not None:
            #            if packer_str != "":
            #                packer_str += ";"
            #            packer_str += res.split(":")[0]
            #    output.append("%s:%s:%s" % (assets, data_type, packer_str))
        return output

    def appCheck2(self, app_df, app_md5, mutual_dict, threshold):
        output = []
        asset_vector = []
        packer_vector = []
        type_vector = []
        vector_hash = []
        assets = None
        data_type = None
        output = self.checkRow(app_df, mutual_dict, threshold)
        for line in output:
            asset_vector.append(line.split(":")[0])
            type_vector.append(line.split(":")[1])
            packer_vector.append(line.split(":")[2])
            vector_hash.append(line.split(":")[3])
        # Put into dataframe
        if len(packer_vector) > 0:
            packer_counter = Counter(packer_vector)
            most, c = packer_counter.most_common()[0]
        else:
            most = "None"
        row = {
            'app_md5': app_md5, 
            'assets_vector': asset_vector, 
            'packer_vector': packer_vector, 
            'type_vector': type_vector, 
            'tested': app_df.iloc[0]['packer'],
            'predict': most,
            'vector_hash': vector_hash
        }
        print(row)
        return row

    def appCheck(self, input_df, mutual_dict, threshold):
        ## Check based on app_md5, recommend on single test
        from IPython.display import clear_output
        total_app = len(input_df['app_md5'].unique())
        pbar = tqdm(total=total_app)
        if self.cacheFlg:
            self.loadCache()
        count = 0
        out_df = pd.DataFrame()
        for app in input_df['app_md5'].unique():
            print("Calculate For App: %s" % app)
            app_df = input_df[input_df['app_md5'] == app]
            print("Assets number: %d" % len(app_df.md5.unique()))
            if len(app_df.md5.unique()) > 40:
                continue
            if self.cacheFlg:
                self.loadAppCache(app)
            row = self.appCheck2(app_df, app, mutual_dict, threshold)
            out_df = out_df.append(row, ignore_index=True)
            # Store
            if self.cacheFlg:
                self.storeAppCache(app)
            if count % 5 == 0:
                clear_output()
            pbar.update(1)
            count += 1
            if count > 0:
                break
        pbar.close()
        return out_df
    
    def assetsCheck(self, input_df, mutual_dict, threshold):
        import json
        from IPython.display import clear_output

        if self.cacheFlg:
            self.loadCache()
        ## Check based on vector_hashes, recommend for batch
        total_assets = len(input_df['vector_hash'].unique())
        pbar = tqdm(total=total_assets)
        count = 0
        out_df = pd.DataFrame()
        for app in input_df['vector_hash'].unique():
            app_df = input_df[input_df['vector_hash'] == app]
            row = self.appCheck2(app_df, app, mutual_dict, threshold)
            out_df = out_df.append(row, ignore_index=True)
            pbar.update(1)
            count += 1
            if count % 50 == 0:
                clear_output()
        pbar.close()
        if self.cacheFlg:
            self.storeCache()
        
        return out_df

    def classify(self, input_df, mutual_dict, threshold=10, assetCheck=False):
        from IPython.display import clear_output

        out_df = pd.DataFrame()
        if assetCheck:
            # ONLY RUN THIS ONE FOR GENERATE THE ASSETS HASHES (CACHE)
            self.checkType = 0
            out_df = self.assetsCheck(input_df, mutual_dict, threshold)
        else:
            self.checkType = 1
            out_df = self.appCheck(input_df, mutual_dict, threshold)
        return out_df

In [13]:
def statistic(packer_test_df, normal_test_df=None):
    p_np = packer_test_df[packer_test_df['predict'] == "None"]  # Packer - non predict
    p_p = packer_test_df[packer_test_df['predict'] != "None"]  # Packer - predicted
    if normal_test_df is not None:
        n_np =normal_test_df[normal_test_df['predict'] == "None"]   # Normal - non predict
        n_pp = normal_test_df[normal_test_df['predict'] != "None"]   # Normal - predicted
        print("Normal In Test: %d" % len(normal_test_df.index))
        print("Correct Normal Apps: %d " % len(n_np.index))
        normal_assets=0
        for i,r in n_np.iterrows():
            normal_assets += len(r['assets_vector'])

    wrong_predict = packer_test_df[packer_test_df['predict'] != packer_test_df['tested']]
    wrong_value_predict = wrong_predict[wrong_predict['predict'] != "None"]
    correct_predict = packer_test_df[~packer_test_df.app_md5.isin(wrong_predict.app_md5)]
    #UPX_df = wrong_predict[wrong_predict['tested'].str.contains("UPX")]

    print("Packer In Test: %d" % len(packer_test_df.index))
    print("Correct Predict: %d" % len(correct_predict.index))
    print("Wrong Predict: %d" % len(wrong_predict.index))
    print(" + Wrong Value Predict: %d" % len(wrong_value_predict.index))
    print(" + Missed: %d" % len(p_np.index))
    #print(" + Wrong Predict (UPX App): %d"  % len(UPX_df.index))

    wrong_app = 0

    for app in wrong_predict['app_md5'].unique():
        app_df = wrong_predict[wrong_predict['app_md5'] == app]
        wrong_app += 1

    count_correct_assets=0
    count_incorrect_assets=0

    for i,r in correct_predict.iterrows():
        count_correct_assets += len(r['assets_vector'])
    for i,r in wrong_predict.iterrows():
        count_incorrect_assets += len(r['assets_vector'])

    print("Correct assets: %d" % count_correct_assets)
    print("Incorrect assets: %d" % count_incorrect_assets)

In [14]:
# CODE - OUTPUT CLASSIFICATION RESULT BASED ON MUTUAL INFORMATION AND TEST DF
# TOOK 6HOURS TO RUN FOR 34067 ASSETS AROUND 1-3s EACH ASSETS
#PACKER_TEST_DF = os.path.join(ASSETS_DIR, "packer_test_df_output_27012019_v5.pickle")
#packer_test_df = pd.DataFrame()
#if os.path.isfile(PACKER_TEST_DF):
#    with open(PACKER_TEST_DF, "rb") as fd:
#        packer_test_df = pd.read_pickle(fd)
#else:
    ## CACHE DUPLICATED TEST 
    #mi_obj = MutualInformationClassification()
    #dup_test_df = mi_obj.classify(reduced_test_df, mutual_dict, 10, True)
    
mi_obj = MutualInformationClassification()
packer_test_df = mi_obj.classify(merged_test_df, mutual_dict, 10)

  1%|          | 1/195 [02:00<6:29:08, 120.35s/it]


In [15]:
PACKER_TEST_DF = os.path.join(ASSETS_DIR, "packer_test_df_output_27012019_v5.pickle")
packer_test_df.to_pickle(PACKER_TEST_DF)

In [16]:
statistic(packer_test_df)

Packer In Test: 1
Correct Predict: 0
Wrong Predict: 1
 + Wrong Value Predict: 0
 + Missed: 1
Correct assets: 0
Incorrect assets: 0


In [17]:
wrong_predict = packer_test_df[packer_test_df['predict'] != packer_test_df['tested']]

# Test on Normal dataset
final_normal_df

In [19]:
#if os.path.isfile(os.path.join(ASSETS_DIR, "normal_test_df_27012019_good.pickle")):
#    with open(os.path.join(ASSETS_DIR, "normal_test_df_27012019_good.pickle"), "rb") as fd:
#        normal_test_df = pd.read_pickle(fd)
#else:
final_normal_df = feature_model.refineDF(final_normal_df)
final_normal_df = feature_model.createHashVector(final_normal_df)
NORMAL_TEST_DF = os.path.join(ASSETS_DIR, "normal_test_df_output_27012019_v5.pickle")
if os.path.isfile(NORMAL_TEST_DF):
    with open(NORMAL_TEST_DF, "rb") as fd:
        normal_test_df = pd.read_pickle(fd)
else:
    mi2_obj = MutualInformationClassification()
    mi2_obj.unsetCache()
    normal_test_df = mi2_obj.classify(final_normal_df, mutual_dict, 10)
    
md5_normal_list = []
with open(os.path.join(BASE_DIR, "md5_list_normal_integrated.pickle")) as data_file:    
        md5_normal_list = json.load(data_file)

count_none = 0
count_packer = 0
for md5 in md5_normal_list:
    target_normal_df = normal_test_df[normal_test_df['app_md5'] == md5]
    if target_normal_df.empty:
        count_none += 1
    else:
        count_packer += 1

In [21]:
# Output
statistic(packer_test_df)
 
# We got the number from previous steps (statistic), the algorithm to got this one is from the paper

tn = 398 # True Neg
fn = 11 # False Neg
tp = 187 # True Pos
fp = 8 # False Pos
print("Normal Dataset: %d" % len(md5_normal_list))
print("  + Non-packer: %d " % count_none)
print("  + Packer: %d " % count_packer)
p_accuracy = ((tn + tp) / (409+195))*100
p_precision = (tp/195)*100
print("Packer Accuracy: %s" % p_accuracy)
print("Packer Precision: %s" % p_precision)

n_precision = (count_none/201)*100
print("Normal Precision: %s" % n_precision)




Packer In Test: 1
Correct Predict: 0
Wrong Predict: 1
 + Wrong Value Predict: 0
 + Missed: 1
Correct assets: 0
Incorrect assets: 0
Normal Dataset: 201
  + Non-packer: 194 
  + Packer: 7 
Packer Accuracy: 96.8543046357616
Packer Precision: 95.8974358974359
Normal Precision: 96.51741293532339
